In [5]:
%load_ext chime
import chime
import rasterio as rio
import os
from glob import glob
from rasterio.windows import from_bounds
import numpy as np
from shapely.geometry import box

The chime extension is already loaded. To reload it, use:
  %reload_ext chime


In [13]:
with rio.open(r"R:\ProjectData\PAG2019\EPCExtent_30cm\Elevation_80cmNPS\DSM_80cm\RGB_438_530_190615_1822_0_528096_2848_DSM80cm.tif") as dsm:
    dsmbnds = box(*dsm.bounds)
with rio.open(loc_dem_2015) as dem:
    dembnds = box(*dem.bounds)
print(dsmbnds.intersection(dembnds).is_empty)

True


In [20]:
chime.theme("material")
loc_dem_2015 = r"E:\EPC_DEM_2015.vrt"
loc_dsms = glob(r"R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm/*.tif")
loc_hags = "R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/HAG_2015"
os.makedirs(loc_hags, exist_ok=True)

In [36]:
with rio.open(loc_dem_2015) as dem_src:
    dem2015_box = box(*dem_src.bounds)

    for i, loc_dsm in enumerate(loc_dsms[:]):
        
        ohag = os.path.join(loc_hags, os.path.basename(loc_dsm).replace(".tif","_HAG.tif"))
        if os.path.exists(ohag):
            continue
        with rio.open(loc_dsm) as dsm_src:
            kwargs = dsm_src.profile
            bnds = dsm_src.bounds
            dsm_box = box(*bnds)
            mutual_area = dsm_box.intersection(dembnds)
            if mutual_area.is_empty:
                #print("No intersection. Skipping")
                continue
            else:
                print(f"{i} - {loc_dsm}")
                dsm_window = from_bounds(*mutual_area.bounds, transform=dsm_src.transform).round_offsets()
                hag_transform = rio.windows.transform(dsm_window, dsm_src.transform)
                dsm = dsm_src.read(1, window=dsm_window)
                dsm_mask = dsm==dsm_src.nodata
                dsm[dsm_mask] = np.nan
                kwargs = dsm_src.profile

                dem = dem_src.read(1, window=from_bounds(*mutual_area.bounds, transform=dem_src.transform).round_offsets(), out_shape=(dsm.shape))
                dem_mask = dem==dem_src.nodata
                dem[dem_mask] = np.nan
        
        mask = dem_mask & dsm_mask
        hag = dsm - dem
        hag[mask] = np.nan
        if not np.isnan(np.nanmax(hag)):
            kwargs.update(transform=hag_transform, width=dsm_window.width, height=dsm_window.height)
            if not os.path.exists(ohag):
                print(f"\tWriting out HAG {ohag}")
                with rio.open(ohag, 'w', **kwargs) as dst:
                    dst.write(hag, 1)
        #else:
        #    print(f"Removing {loc_dsm}")
        #    loc_dsms.remove(loc_dsm)
        
%chime

438 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_128480_2592_DSM80cm.tif
439 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_134624_2592_DSM80cm.tif


C:\Users\BenJames\AppData\Local\Temp/ipykernel_14652/3049977438.py:33: RuntimeWarning: All-NaN slice encountered
  if not np.isnan(np.nanmax(hag)):


440 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_140768_2592_DSM80cm.tif
441 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_146912_2592_DSM80cm.tif
442 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_153056_2592_DSM80cm.tif
443 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_159200_2592_DSM80cm.tif
444 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_165344_2592_DSM80cm.tif
445 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_171488_2592_DSM80cm.tif
446 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_177632_2592_DSM80cm.tif
448 - R:/ProjectData/PAG2019/EPCExtent_30cm/Elevation_80cmNPS/DSM_80cm\RGB_420_522_190616_1547_0_183776_2592_DSM80cm.tif
449 - R:/ProjectData/PAG2019/EPC

In [51]:
for i in range(brawleyrillto.columns.size):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [58]:
for i in range(brawleyrillto.columns.size):
    if i<=25:
        print(i, alphabet[i])
    else:
        print("A"+alphabet[i-26] )

0 a
1 b
2 c
3 d
4 e
5 f
6 g
7 h
8 i
9 j
10 k
11 l
12 m
13 n
14 o
15 p
16 q
17 r
18 s
19 t
20 u
21 v
22 w
23 x
24 y
25 z
Aa
Ab
Ac
Ad
Ae
Af
Ag
Ah
Ai
Aj


In [60]:
import pandas as pd
alphabet = [chr(alpha) for alpha in range(97, 123)]
brawleyrillto = pd.read_csv("C:/Users/BenJames/Downloads/data.csv")
brawleyrillto.columns = [alphabet[i] if i<=25 else "A"+alphabet[i-26] for i in range(brawleyrillto.columns.size) ]

In [69]:
for i, tif in enumerate(brawleyrillto.t.tolist()):
    #with rio.open(tif) as src:
    #    kwargs = src.profile
    ofile = f"R:/ProjectData/PAG2019/1mDEMs/{os.path.basename(tif)}"
    if not os.path.exists(ofile):
        print(ofile)

R:/ProjectData/PAG2019/1mDEMs/USGS_one_meter_x52y357_AZ_Eastern_PimaCO_2015.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x52y358_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_one_meter_x52y358_AZ_Eastern_PimaCO_2015.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x52y359_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x52y360_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x53y355_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_one_meter_x53y355_AZ_Eastern_PimaCO_2015.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x53y356_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_one_meter_x53y356_AZ_Eastern_PimaCO_2015.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x53y357_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x53y358_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x53y359_AZ_BrawleyRillito_FEMA_2018.tif
R:/ProjectData/PAG2019/1mDEMs/USGS_1m_x54y355_AZ_Brawley